# Extra Trees classifier introduction
* this is  a ensemble bagging  method coming from sklearn



In [ ]:
#For Kaggle
#date=''

#For WL data
source='WL'
date='20241118'

saveImage=False

print('done')

In [ ]:
import sys
import os
from importlib import reload
fpath = os.path.join('..//scripts')
sys.path.append(fpath)

import warnings
warnings.filterwarnings('ignore')

#loading internal scripts
import frauddetection as fd
import sourcedata as sd
import dataimport as di
import result as resultMd
import sampling as sp
reload(fd)
reload(sd)
reload(di)
reload(resultMd)
reload(sp)

print('done')

In [ ]:
# Hyperparameters Results

n_estimatorsFound=115
max_depthFound=28
min_samples_splitFound=2
min_samples_leafFound=1



print('done')

In [ ]:
import pandas as pd

dfTrx = pd.read_csv('../data/cleaned/'+source+'export'+date+'.csv')
dfTrx.head(5)

In [ ]:
from sklearn.model_selection import train_test_split

predictors = fd.getPredictors(dfTrx)
TEST_SIZE = 0.20 # test size using_train_test_split
RANDOM_STATE = 0
x_train0, x_test, y_train0, y_test = train_test_split(dfTrx[predictors], dfTrx['Class'], test_size = TEST_SIZE, 
                                                        stratify= dfTrx['Class'],
                                                        random_state = RANDOM_STATE)

x_train, y_train = sp.randomUnderSampling(x_train0, y_train0,1/100)


# Test with default hyperparameters

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
from datetime import datetime

then= datetime.now()
predictors = fd.getPredictors(dfTrx)

modelClf = ExtraTreesClassifier(random_state=42)
modelClf.fit(x_train, y_train)
predsTrain = modelClf.predict(x_train)
predsTest = modelClf.predict(x_test)

now = datetime.now()
duration= now - then
duration_in_s = duration.total_seconds()
print("Duration ",duration_in_s)
#resultMd.update_time_response_result('07-sklearn.ensemble.a-bagging','ExtraTreesClassifier','4-After undersampling', duration_in_s)
f1,mcc,roc=fd.print_scores(y_test, predsTest,'all')
#resultMd.update_performance_test_result('07-sklearn.ensemble-a-bagging','ExtraTreesClassifier','4-After undersampling', f1,mcc,roc)

fd.print_scores(y_train, predsTrain,'f1', False)
fd.print_scores(y_test, predsTest,'f1')
fd.show_importance(modelClf, predictors)
fd.show_confusion_matrix(y_test, predsTest)


# Scaling choice
with ensemble algorithm the scaling has no influence

# Hyperparameters tuning

In [ ]:
%%script false

from sklearn.ensemble import ExtraTreesClassifier
from scipy.stats import randint

modelClf = ExtraTreesClassifier(random_state=42)
dic_param={
    'n_estimators': randint(90,130),
    'max_depth': randint(17,42),
    'min_samples_leaf': randint(1,2),
    'min_samples_split': randint(2,3)
}
res=fd.hyperparameterSelectionRandomizedSearchCVSampling(modelClf, dic_param, 'f1', x_train, y_train)
print(res)

#{'max_depth': 33, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 111}
#0.5393388747404793
#score   0.9937167713871435
#scoref1 0.9937167713871435

#{'max_depth': 28, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 121}
#0.5367399364574297
#score   0.9718518518518519
#scoref1 0.9718518518518519


In [ ]:
%%script false
from sklearn.ensemble import ExtraTreesClassifier

modelClf = ExtraTreesClassifier(random_state=42)

dic_param={
    'n_estimators': [110,112,115,117,120],
    'max_depth': [27,28,29],
    'min_samples_leaf': [1],
    'min_samples_split': [2]
}
res=fd.hyperparameterSelectionGridSearchCVSampling(modelClf, dic_param, 'f1', x_train, y_train)
print(res)



#{'max_depth': 28, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 115}
#0.5382976352466755
#scoref1 0.9723865877712031



In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
from datetime import datetime

then= datetime.now()
predictors = fd.getPredictors(dfTrx)



modelClf = ExtraTreesClassifier(random_state=42)
parameters={'n_estimators':n_estimatorsFound,
    'max_depth': max_depthFound, 
            'min_samples_leaf': min_samples_leafFound, 
            'min_samples_split': min_samples_splitFound,
           }

modelClf.set_params(**parameters)

modelClf.fit(x_train, y_train)
predsTrain = modelClf.predict(x_train)
predsTest = modelClf.predict(x_test)

now = datetime.now()
duration= now - then
duration_in_s = duration.total_seconds()
print("Duration ",duration_in_s)
#resultMd.update_time_response_result('07-sklearn.ensemble.a-bagging','ExtraTreesClassifier','2-After tuning', duration_in_s)
f1,mcc,roc=fd.print_scores(y_test, predsTest,'all')
#resultMd.update_performance_test_result('07-sklearn.ensemble.a-bagging','ExtraTreesClassifier','2-After tuning', f1,mcc,roc)

fd.print_scores(y_train, predsTrain,'f1', False)
fd.print_scores(y_test, predsTest,'f1')
fd.show_importance(modelClf, predictors)
fd.show_confusion_matrix(y_test, predsTest)
fd.show_prediction_graph(modelClf, x_test,y_test,'../imgs/FD07A-BaggingClassifierClassifierProbaHistogram' if saveImage else None)



In [ ]:
files = fd.getAllFiles()

range = []
f1s = []
rocs = []
loop =0
for file in files:
    loop=loop+1
    range.append(loop)
    print(file)
  
    dfTrx = pd.read_csv('../data/cleaned/'+source+file)
    predictors = fd.getPredictors(dfTrx)

    preds = modelClf.predict(dfTrx[predictors])

    f1,mcc,roc= fd.print_scores(dfTrx['Class'], preds,'All', True)
    #fd.show_importance(modelClf,predictors)
    fd.show_confusion_matrix(dfTrx['Class'], preds)
    f1s.append(f1)
    rocs.append(roc)

fd.plt_train_test(range, f1s)
resultMd.update_performance_nextdays_result('07-sklearn.ensemble.a-bagging','ExtraTreesClassifier','4-After undersampling', f1s[0],f1s[1],f1s[2],f1s[3],rocs[0],rocs[1],rocs[2],rocs[3])

